# Object-Orientied Programming in Python - Working with Classes

## Contents
- Python Scopes & Namespaces
- Class Definition
- Inheritance
- Multiple Inheritance




## Python Scopes & Namespaces

https://docs.python.org/3/tutorial/classes.html

Firstly,we need to understand the rules about scopes and namespace in Python.

- **Namespace**: a mapping from names to objects. Most namespaces are currently implemented as Python dictionaries, but that's normally not noticeable (this may also change in future). 

Examples of namespaces: the set of built-in names; the global names in a module; the local names in a function invocation.

The important thing to know about *namespaces* is that there is absolutely NO relation between names in differnet namespace. For instance, two different modules may both define a function `maximize` without confusion - Users of the modules must prefix it with the module name.

Namespaces are created at different moments and have different lifetimes. The namespace containing the built-in names is created when the Python interpreter starts up, and is never deleted. The global namespace for a module is created when the module definition is read in. The local namespace for a function is created when the function is called, and deleted when the function returns or raises an exception that is not handled within the function.

- **Scope**: a textual region of a Python program where a namespace is directly accessible. 

A special quirk of Python is that - if no `global` statement is in effect, assignments to names always go into the innermost scope. Assignments do not copy data - they just bind names to objects. The same is true for deletions: the statement `del x` removes the binding of `x` from the namespace referenced by the local scope. In fact, **all operations that introduce new names use the local scope**: in particular, `import` statements and function definitions bind the module or function name in the local scope.

The `global` statement can be used to indicate that particular variables live in the global scope and should be rebound there; the `nonlocal` statement indicate that particular variables live in an enclosing scope and should be rebound there.

The example below demonstrates how to reference the different scopes and namespace, and how `global` and `nonlocal` affect variable binding (**Note:** `nonlocal` may not work properly in Python 2):

```python
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)
```

RESULT:
```
After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam
```

## Class Definition

Class definitions, like function definitions, must be executed before they have any effect.

In practice, the statements inside a class definition will usually be function definitions, but other statements are allowed and sometimes useful as well.

When a class definition is entered, a new namespace is created, and used as the local scope. Thus, all assignments to local variables go into this new namespace.

There are two kinds of operations on class objects:

- attribute references
- instantiation

### Attribute references

In [1]:
class MyClass:
    i = 12345
    
    def f(self):
        return('hello world')
    
x = MyClass()
print x.i
print x.f
print x.f()

12345
<bound method MyClass.f of <__main__.MyClass instance at 0x106906200>>
hello world


### Instantiation

The instantiation operation ("calling" a class object) creates an empty object. Many classes like to create objects with instances customized to a specific initial state. Therefore a class may define a special method `__init__()`. When a class defines an `__init__()` method, class instantiation automatically invokes `__init__()` for the newly-created class instance.

In [2]:
class Complex:
    def __init__(self, realpart, imagepart):
        self.r = realpart
        self.i = imagepart
        
x = Complex(3.0, -4.5)
print x.r, x.i

3.0 -4.5


There are two kinds of valid attribute names, **data attributes** and **methods**. Note a **method** is a function that "belongs to" an object.

What exactly happens when a **method** is called? We found that x.f() was called without an argument above, even though the function definition for `f()` specified an argument. It's weird.

The answer is: the instance object is passed as the first argument of the function. In the example above, the call `x.f()` is exactly equivalant to `MyClass.f(x)`. In general, calling a method with a list of *n* arguments is equivalent to calling the corresponding function with an argument list that is created by inserting the method's instance object before the first argument.

### Class and Instance Variables

Generally speaking, **instance variables** are for data unique to each instance and **class variables** are for attributes and methods shared by all instances of the class. 

In [3]:
class Dog:
    
    kind = 'canine' # class variable shared by all instances
    
    def __init__(self, name):
        self.name = name # instance variable unique to each instance
        
d = Dog("Fido")
e = Dog("Buddy")

print d.kind
print e.kind

print d.name
print e.name

canine
canine
Fido
Buddy


**NOTE**: shared data, like class variable can have possibly surpring effects with involving** *mutalbe* **objects like *lists* and *dictionaries*. For example, the tricks list in the following code shoudl not be used as a class variable because just a single list would be shared by all Dog instances:

In [4]:
class Dog:

    tricks = []             # mistaken use of a class variable

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)
        
d = Dog("Fido")
e = Dog("Buddy")
d.add_trick("roll over")
print e.tricks # unexpectedly shard by all instances

['roll over']


Correct design of the class should use an instance variable instead.

In [5]:
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)
        
d = Dog("Fido")
e = Dog("Buddy")
d.add_trick("roll over")
print d.tricks
print e.tricks

['roll over']
[]


Sometimes we may want to check the class (or called `type`) of an object, we can use the codes below

In [6]:
print type(e)
print e.__class__

<type 'instance'>
__main__.Dog


## Inheritance

A language feature would not be worthy of the name "class" without supporting inheritance. The syntax for a derived class definition looks like this:

```python
class DerivedClassName(BaseClassName):
    <statement-1>
    .
    .
    .
    <statement-N>
```

The name *`BaseClassName`* must be defined in a scope containing the derived class definition. 

Derived classes may override methods of their base classes. Because methods have no special privileges when calling other mehtods of the same object, a method of a base class that calls another method defind in the same base class may end up calling a method of a derived class that overrides it.

In [7]:
class BaseClass():
    
    def __init__(self, value):
        self.value = value

    def present_value(self):
        print(self.value)
        
        
class DerivedClass(BaseClass):
    
    # override the method
    def present_value(self):
        print(self.value * 2)
        
a = BaseClass(5)
b = DerivedClass(5)

a.present_value()
b.present_value()

5
10


## Multiple Inheritance

Python supports a form of **multiple inheritance** as well. A class definition with multiple base classes looks like this:

```python
class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>
```

For most purposes, in the simplest cases, we can think of the search for attributes inherited from a parent class as **depth-first**, **left-to-right**, **not searching twice in the same class where there is an overlap** in the hierarchy. Thus, if an attribute is not found in `DerivedClassName`, it's searched for in `Base1`, then (recursively) in the base classes of `Base1`, and if it was not found there, it was searched for in `Base2`, and so on.

In [8]:
class Base1():
    value_a = 1
    
class Base2():
    value_a = 11
    value_b = 8
    
class Base3():
    value_b = 88
    value_c = 99
    
class DerivedClass(Base1, Base2, Base3):
    pass

test = DerivedClass()

print test.value_a
print test.value_b
print test.value_c

1
8
99


In fact it may be slightly more complex than this. The method resolution order changes dynamically to support cooperative calls to `super()`. This approach is known in some other multiple-inheritance languages as call-next-method and is more powerful than the super cal found in single-inheritance languages. But these will not be covered here.